### Compare gtdb and mags
If (and I think this is true) new MAGs and undereplicated GTDB provide almost the same coverage, then how much new genomic content did we just uncover? Did we uncover anything new? How many of the kmers in MAGs are also in GTDB (all)? If it's almost everything, that would suggest that most of the sequence diversity is already captured in GTDB, but people aren't using it. That would also mean that no new MAGs have to be created, because that is not helping. Instead we would have to look for a different way of finding more kmers belonging to (yet) unknown species. 


How to test:
- Make pangenomedb of gtdb all
- Subtract MAGs (all) from GTDB. 

Threshold of 0 recovers between 70-90% 

### Species level pangenomedb
With a species lvl pangenomedb and threshold of 0: What do we get? What species are abundant. 
What species are now better represented than before in pig gut metaGs? 
Which ones now have higher f_unique weighted? How many of that species did we assemble?


In [ ]:
## Redo taxonomy for MAGs with GTDB r226
# Then redo making a pangenomedb and a taxonomy file. See below.
# there are 338 spp that do not have species tax: Will remove from db for now. 

# will need to subset and resketch though/..
# made manysketch_all_taxed.csv for sketching
# merge signatures. 
srun --account=ctbrowngrp -p med2 -J calc-pres -t 4:00:00 -c 24 --mem=80gb --pty bash

# run the script
mamba activate branchwater-skipmer

# create db
sourmash scripts manysketch -p dna,k=21,k=31,k=51,scaled=1000,abund \
-c 24 manysketch_all_taxed.csv -o MAGs.all_taxed.zip


# make to pangenome db
mamba activate pangenomics_dev
sourmash scripts pangenome_createdb \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.all_taxed.zip \
-t 250828_sourmash_tax.gtdbrs226.csv \
-k 31 --scaled 10000 -o sra_mags.pangenomedb_speciestaxed.10k.zip 



# tax prep
sourmash tax prepare --taxonomy \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226.lineages.csv \
250828_sourmash_tax.gtdbrs226.csv \
-o 250903_gtdb_sraMAGs.lineages.csv

## Rerun gather
would be nice if we have the singlem comparison files first. Run gather x those?

In [ ]:
# redo gather
# The results for th
mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 24 --rerun-incomplete -k -s gather_pangenomedb.smk -n

In [ ]:
# create the pangenome db 

# Can I give this 2 dbs?
# Doesnt matter: Gather can take 2 rocksdbs 
# Can then use the concatenated tax file 

# in results/gtdb_pangenomedb
mamba activate pangenomics_dev
sourmash scripts pangenome_createdb \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226.k21.rocksdb \
-t /group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226.lineages.csv \
-k 31 --scaled 10000 -o gtdb-rs226.pangenomedb_species.10k.zip


# one for mags
mamba activate pangenomics_dev
sourmash scripts pangenome_createdb \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.all_k31.rocksdb \
-t 250807_tax_for_sourmash.csv \
-k 31 --scaled 10000 -o sra_mags.pangenomedb_species.10k.zip 



In [ ]:
# combining the GTDB with MAGs taxonomy files
sourmash tax prepare --taxonomy \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226.lineages.csv \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/gtdb_pangenomedb/250807_tax_for_sourmash.csv \
-o 250807_gtdb_sraMAGs.lineages.db



## Run gather, then tax annotate
- Gather against GTDB species 
- Gather against GTDB + MAGs
- Use threshold of 0

In [ ]:
# Run gather in a snakemake then run sourmash tax??
srun --account=ctbrowngrp -p bmm -J calc-pres -t 8:00:00 -c 120 --mem=120gb --pty bash

# run the script
mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 120 --rerun-incomplete -k -s gather_pangenomedb.smk -n


# Make a species level pangenome db 
- One with just GTDB and one with GTDB + MAGs
- Gives us an idea of what species are now better represented


In [ ]:
# did colton already have db? 
 find /some/path -name \*substring\* -type d\
find /group/ctbrowngrp3/baumlerc -name \*gtdb\* -type d

~/dissertation-project/fastgather-test/query_dbs/gtdb-rs220-k31.species.zip

In [ ]:

# then for sketching
srun --account=ctbrowngrp -p med2 -J sketch -t 24:00:00 -c 4 --mem=100gb --pty bash

srun --account=ctbrowngrp -p med2 -J sketch -t 1:00:00 -c 1 --mem=50gb --pty bash

# then also make a pangenome db bc its faster 
mamba activate pangenomics_dev
sourmash scripts pangenome_merge \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226.k21.rocksdb \
-k 21 --scaled 10000 -o gtdb-rs226.pangenomedb.10k.zip


# also maybe one for reps (wait with running gather until finised)
sourmash scripts pangenome_merge \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226-reps.k21.rocksdb \
-k 21 --scaled 10000 -o gtdb-rs226-reps.pangenomedb.10k.zip

could we do sig merge?

In [ ]:
find /group/ctbrowngrp* -name \*baumler\* -type d